## Описание корпуса и стандарта разметки

In [30]:
from pprint import pprint

In [31]:
raw_text = ''
with open('pushkin.txt', 'r', encoding='utf-8') as raw_file:
    raw_text = raw_file.read()

# Как показала практика прошлого года, текст лучше скармливать
# со знаками препинания. Но перенос строки мы всё же
# заменим на пробел

raw_text = raw_text.replace('\n', ' ')

In [32]:
answers = []
with open('pushkin POS.txt', 'r', encoding='utf-8') as answer_file:
    words = answer_file.readlines()
    for word in words:
        answers.append(word.strip('\n').split(' '))

## Разметить текст тремя POS-тэггерами + превести к стандартному виду

### spacy

In [16]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import spacy
from spacy.lang.ru.examples import sentences
!python -m spacy download ru_core_news_sm
nlp = spacy.load("ru_core_news_sm")

2022-10-11 16:45:07.853070: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 28.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [33]:
spacy_answer_raw = []

doc = nlp(raw_text)
for t in doc:
    spacy_answer_raw.append([t.text, t.pos_])

In [34]:
# приведём разметку к единому стандарту

sp_tags = {'NOUN':'S', 'PROPN':'S', 'ADJ':'A', 'VERB':'V',
           'PART':'ADV', 'SCONJ':'CONJ', 'CCONJ':'CONJ', 
           'ADV':'ADV', 'ADP':'PR', 'DET':'PRO', 'NUM':'NUM',
           'PRON':'PRO'}

spacy_answer = []

for sar in spacy_answer_raw:
    if sar[1] != 'PUNCT':
        spacy_answer.append([sar[0], sp_tags[sar[1]]])

## Оценка accuracy

In [35]:
print(len(spacy_answer))
print(len(answers))

233
223


In [37]:
out_tags = ['PRO', 'NUM', 'CAT', 'COMP'] # не учитываем

Spacy разделял словоформу с дефисом на три части: первую, дефис и вторую. Будем оценивать правильность по тегу последней.

In [39]:
gap = 0
sp_t_score = 0 # количество правильных

for i, a in enumerate(answers):
    # сверяем, что мы смотрим на одно и то же 
    # mystem разделил 3 формы с пробелами ()
    if a[0] != spacy_answer[i+gap][0]:
        gap += 2
        
    # сверяем сами теги
    if a[1] not in out_tags:
        if a[1] == spacy_answer[i+gap][1]:
            sp_t_score += 1

print('Accuracy for spacy: ')
print(str(sp_t_score/223))

Accuracy for spacy: 
0.7713004484304933


Вывод: spacy справляется на порядок хуже mystem'a и pymorphy.